<a href="https://colab.research.google.com/github/delhian/NIR/blob/main/%D0%9D%D0%98%D0%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install umap-learn > /dev/null
!pip install transformers > /dev/null

In [75]:
import re
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import random
import umap.umap_ as umap
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm

from bokeh.models import ColumnDataSource, LabelSet, Legend
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [12]:
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# name = '/content/drive/MyDrive/Anthony & Margroff - K1 - Dragon\'s Gold.txt'
name = '/content/drive/MyDrive/text.txt'
with open(name, 'r', ) as file:
    text = file.read()
print(len(text))

541415051


In [15]:
# sub = '[\?\.\!]([^\?\.\!]*?[^a-z-]'+'red'+'[^a-z-][^\?\.\!]*?[\?\.\!])'

# prog = re.compile(sub)

# for ind, m in enumerate(re.finditer(prog, text.lower())):
#   print('**********')
#   print(text[m.start() + 1: m.end()])
#   print(m.group(0))
#   print('**********')
#   if ind  == 5:
#     break


In [16]:
%%time

phrase_list = ['pink', 'crimson','red','blue','brown' ,'orange', 'grey','green', 'gray', 'yellow', 'white', 'black','purple']
# phrase_list = ['gray']

def f_dfmake(text, phrase_list, n_max):

  # phrase_list = ['red',]
  emb = []
  for token in phrase_list :
    # print(token)
    emb_token = []
    sub = '[\?\.\!]([^\?\.\!]*?[^a-z-]'+token+'[^a-z-][^\?\.\!]*?[\?\.\!])'
    prog = re.compile(sub)
    iter = re.finditer(prog, text.lower())
    for ind, seq in enumerate(iter):
      txt = text[seq.start() + 1: seq.end()]
      txt_lower = seq.group(0)
      # print(seq.group(0))
      seq_tokenized = tokenizer.tokenize(txt_lower)
      seq_tokenized_length = len(seq_tokenized)
      if seq_tokenized_length > 512: ## too long sequence!
        # print(f'seq_tokenized_length too long: {seq_tokenized_length} !')
        continue
      # print(seq_tokenized)
      ind = seq_tokenized.index(token)
      encoded_input = tokenizer(txt_lower, return_tensors='pt')
      out = model(**encoded_input)[0][0,1:-1]
      out = out[ind].detach().numpy()
      emb_token.append([token, txt , out])
      # print(len(emb_token))
      if len(emb_token)  == n_max:
        break
    emb += emb_token
  emb = pd.DataFrame(emb, columns = ['token', 'txt', 'word_embedding'])
  print(emb.token.value_counts(dropna=False))
  return emb

n_max = 20
emb = f_dfmake (text, phrase_list, n_max)

Token indices sequence length is longer than the specified maximum sequence length for this model (1186 > 512). Running this sequence through the model will result in indexing errors


black      20
crimson    20
orange     20
blue       20
red        20
yellow     20
brown      20
pink       20
purple     20
white      20
green      20
gray       20
grey       20
Name: token, dtype: int64
CPU times: user 1min 20s, sys: 3.27 s, total: 1min 23s
Wall time: 1min 23s


In [67]:
emb.head()

,token,txt,word_embedding,x,y,index
0,pink,"\n\tEdorion drew a deep breath, puffing his pi...","[-0.8603322, -0.38904473, 0.104271926, -0.4258...",8.525400,10.856982,0
1,pink,There were pink ones in between to show that ...,"[-0.15525538, -0.18555067, 0.071046904, -0.084...",8.911393,11.932076,1
2,pink,"\n""Morning, Jack,"" Cathy said, entering the ki...","[-0.26438084, -0.62798965, 0.19889872, -0.0676...",9.085867,12.192217,2
3,pink,She was really clean for the first time in ou...,"[-0.6256314, -0.8682387, 0.35934496, -0.477517...",8.985619,11.592886,3
4,pink,I'm in the pink. How about yourself?,"[-0.62260383, -0.5831793, -0.08096335, -0.0307...",8.832524,11.816238,4


In [38]:
emb.token.unique()

array(['pink', 'crimson', 'red', 'blue', 'brown', 'orange', 'grey',
       'green', 'gray', 'yellow', 'white', 'black', 'purple'],
      dtype=object)

In [116]:
colormap = {'setosa': 'red', 'versicolor': 'green', 'virginica': 'blue'}
colors = [colormap[x] for x in flowers['species']]

In [3]:
[x for x in emb.token if x in list(matplotlib.colors.cnames.keys())]

In [121]:
matplotlib.colors.cnames.keys()

dict_keys(['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond', 'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkgrey', 'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'gray', 'green', 'greenyellow', 'grey', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgreen', 'lightgrey', 'lightpink', 'lightsalmon', 'lights

In [112]:
def f_plot(df, n_neighbors , min_dist, phrase_list):

  arr_proj = umap.UMAP(n_neighbors = n_neighbors, min_dist =min_dist).\
  fit_transform(np.array(df['word_embedding'].values.tolist()))
  
  df['x'], df['y'] = arr_proj[:,0], arr_proj[:,1]

  df['index'] = df.index

  colors = ['pink'] * len(df.token)

  source = ColumnDataSource(df)

  p = figure(title = "Contextualized word embedding: UMAP-projection",
             background_fill_color="#fafafa", plot_width=1500)
  
  p.title.align = 'center'
  

  p.add_layout(Legend(),'right')

  labels = LabelSet(x='x',
                    y='y',
                    text='index',
                    source=source,
                    text_font_size = '12pt')

  
  p.scatter(x='x', y='y', size=10, source = source,
            legend_group='token',\
            fill_alpha=1,
            color='token',
            line_color="black",
            )

  p.add_layout(labels)
  show(p)

In [131]:
f_plot(emb, n_neighbors =15, min_dist = 0.5, phrase_list= phrase_list, )

In [132]:
loi = [153, 50]

tmp = emb[emb['index'].isin(loi)]

for i, row in emb[emb['index'].isin(loi)].iterrows():
  print(row.token)
  print(row.txt)

red
 They said men like that, whether false Dragons or just poor fools the Red Ajah took against, never lived long.
green
 Moiraine knew that "Egwene Sedai of the Green Ajah" was a fraud.
